In [1]:
# Import libraries
import numpy as np 
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Prepare Dataset
# load data

# relative path to npz files
path = 'Measurements'
file_name = 'output_batch_%d.npz'

# Training
# how many frames to load
frames_num = 10
data_frames = []
data_forces = []
for i in range(frames_num):
    file_path = os.path.join(path,file_name %i)
    data = np.load(file_path)
    data_frames.append(data['frames'])
    data_forces.append(data['forces'])

combined_frames = np.concatenate(data_frames, axis=0)
combined_forces = np.concatenate(data_forces, axis=0)

# Test
# file_path = os.path.join(path, file_name %11)
# test_data = np.load(file_path)

In [3]:
# train test split. Size of train data is 10/11 and size of test data is 1/11 
# features_train, features_test, targets_train, targets_test = train_test_split(combined_frames,
#                                                                              combined_forces[:,2],
#                                                                              test_size = (1/11),
#                                                                              random_state = 42) 
# train_test_split NOT RECOMMENDED, because it shuffles temporally dependent data
features_train = combined_frames
targets_train = combined_forces[:,2]
# features_test = test_data['frames']
# targets_test = test_data['forces']
# targets_test = targets_test[:,2]

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train)

# create feature and targets tensor for test set. 
# featuresTest = torch.from_numpy(features_test)
# targetsTest = torch.from_numpy(targets_test)

In [4]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_channels, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 9, kernel_size = 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(kernel_size = 4, stride = 4)
        self.conv2 = nn.Conv2d(9, 18, kernel_size = 5, stride = 1, padding = 2) # If needed
        
        # RNN
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(18 * 16 * 30, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Convolutional layers
        x = self.pool(nn.ELU()(self.conv1(x)))
        x = self.pool(nn.ELU()(self.conv2(x))) # If needed
        
        # Reshape for RNN
        x = torch.reshape(x, (-1, 1, 18 * 16 * 30))  # Reshape to (batch_size, seq_len, input_size)
        print("Size of x:", x.size())  # Print size of x
        
        # RNN
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        #print("Size of h0:", h0.size())  # Print size of h0 - Debugging - Obsolete
        out, hn = self.rnn(x, h0)
        
        # Output layer
        out = self.fc(out[:, -1, :]) 
        return out


In [5]:
# batch_size, epoch and iteration
batch_size = 1000
num_epochs = 5

# Pytorch train and test sets
train = TensorDataset(featuresTrain,targetsTrain)
# test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
# test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_channels = 3  # RGB channels
hidden_dim = 100  # hidden layer dimension
layer_dim = 1     # number of hidden layers
output_dim = 1   # output dimension

model = RNNModel(input_channels, hidden_dim, layer_dim, output_dim)

# Define your loss function
error = nn.MSELoss()

# Define your optimizer
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Training
seq_dim = 50 # Consider the whole batch to be temporally correlated
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    print('~~~BEGINNING OF DATASET~~~')
    for i, (images, labels) in enumerate(train_loader):
        images = images.float()
        # print(images.shape)  # Add this line to check the shape of images - Debugging purposes
        images = images.permute(0, 3, 1, 2)
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(images)
        outputs = torch.squeeze(outputs)
        
        # Calculate loss
        loss = error(outputs, labels.float())
        
        # Backpropagation
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
            
        # Store loss and iteration
        loss_list.append(loss.data)
        # Print Loss
        if count % 1 == 0: # for now print for every iteration
            print('Iteration: {}  Loss: {}'.format(count, loss.data.item()))

~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([1000, 1, 8640])
Iteration: 1  Loss: 1469.69970703125
Size of x: torch.Size([1000, 1, 8640])
Iteration: 2  Loss: 2391.631103515625
Size of x: torch.Size([1000, 1, 8640])
Iteration: 3  Loss: 1126.598388671875
Size of x: torch.Size([1000, 1, 8640])
Iteration: 4  Loss: 6.230687618255615
Size of x: torch.Size([1000, 1, 8640])
Iteration: 5  Loss: 814.4215087890625
Size of x: torch.Size([1000, 1, 8640])
Iteration: 6  Loss: 1036.4716796875
Size of x: torch.Size([1000, 1, 8640])
Iteration: 7  Loss: 344.6268310546875
Size of x: torch.Size([1000, 1, 8640])
Iteration: 8  Loss: 4.114468574523926
Size of x: torch.Size([1000, 1, 8640])
Iteration: 9  Loss: 311.7105712890625
Size of x: torch.Size([1000, 1, 8640])
Iteration: 10  Loss: 616.9794921875
~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([1000, 1, 8640])
Iteration: 11  Loss: 377.4425354003906
Size of x: torch.Size([1000, 1, 8640])
Iteration: 12  Loss: 60.99824905395508
Size of x: torch.Size([10

In [11]:
print(loss_list)
print(np.sqrt(np.mean(loss_list[-10:])))

[tensor(1469.6997), tensor(2391.6311), tensor(1126.5984), tensor(6.2307), tensor(814.4215), tensor(1036.4717), tensor(344.6268), tensor(4.1145), tensor(311.7106), tensor(616.9795), tensor(377.4425), tensor(60.9982), tensor(32.9489), tensor(245.6324), tensor(328.9977), tensor(190.0103), tensor(23.7830), tensor(25.2784), tensor(124.5828), tensor(192.7615), tensor(102.9052), tensor(16.5119), tensor(11.3704), tensor(75.7428), tensor(101.1000), tensor(61.9432), tensor(9.5673), tensor(7.5841), tensor(34.5493), tensor(56.7667), tensor(24.1827), tensor(3.0861), tensor(8.9956), tensor(28.9779), tensor(23.7095), tensor(7.2483), tensor(2.4198), tensor(12.2633), tensor(12.8351), tensor(10.9076), tensor(1.8964), tensor(4.7441), tensor(8.1270), tensor(7.9571), tensor(2.4155), tensor(2.5701), tensor(5.9615), tensor(5.6332), tensor(2.0938), tensor(1.7912)]
2.0782137


In [8]:
print(outputs) # Small sanity check

tensor([-1.4929, -1.5412, -2.1978, -2.1698, -2.3976, -2.0895, -2.1512, -1.4423,
        -2.7281, -2.4608, -2.4110, -3.2214, -1.4060, -3.4376, -4.0289, -4.1386,
        -3.3118, -2.6025, -1.8170, -3.1059, -1.8080, -2.4604, -1.3627, -2.2984,
        -2.3862, -2.5800, -1.4575, -1.7955, -1.8638, -1.5818, -1.4018, -2.4109,
        -1.9005, -2.2340, -3.0158, -1.6643, -2.4143, -2.8597, -2.3950, -1.5725,
        -2.0098, -3.7721, -2.6353, -3.3689, -2.3851, -2.9648, -2.1515, -2.7832,
        -2.0951, -2.2783, -2.1410, -2.0654, -1.8210, -1.6418, -1.4474, -1.7092,
        -1.8115, -2.2257, -1.7392, -1.7605, -2.1168, -2.5319, -2.1835, -3.2506,
        -3.2351, -2.6224, -4.0558, -2.8631, -2.3781, -2.2907, -3.4886, -1.3027,
        -2.7322, -2.0699, -2.2706, -1.6914, -3.3931, -3.0189, -0.7092, -1.1881,
        -0.8689, -1.5976, -1.6396, -3.2310, -1.8672, -3.2908, -2.4769, -2.8107,
        -2.7464, -2.7845, -2.9820, -3.9309, -3.0694, -3.1747, -2.9800, -4.4822,
        -2.4932, -2.5690, -1.7546, -3.18

In [9]:
print(labels.float())

tensor([-1.0797, -0.9949, -0.9238, -0.8581, -0.7988, -0.7402, -0.6864, -0.6413,
        -0.6007, -0.5637, -0.5303, -0.5034, -0.4781, -0.4516, -0.4438, -0.4478,
        -0.4525, -0.4622, -0.4775, -0.4831, -0.4824, -0.4824, -0.4840, -0.4829,
        -0.4796, -0.4784, -0.4745, -0.4723, -0.4696, -0.4686, -0.4738, -0.4907,
        -0.5179, -0.5654, -0.6264, -0.6943, -0.7702, -0.8488, -0.9312, -1.0187,
        -1.1152, -1.2284, -1.3486, -1.4846, -1.6458, -1.8335, -2.0371, -2.2583,
        -2.4803, -2.6908, -2.8747, -3.0401, -3.1835, -3.2991, -3.3773, -3.4140,
        -3.4004, -3.3335, -3.2181, -3.0745, -2.9110, -2.7374, -2.5585, -2.3805,
        -2.1968, -2.0178, -1.8431, -1.6839, -1.5378, -1.3989, -1.2665, -1.1307,
        -0.9906, -0.8688, -0.7793, -0.7269, -0.7026, -0.7065, -0.7217, -0.7390,
        -0.7422, -0.7402, -0.7296, -0.7031, -0.6639, -0.6237, -0.5844, -0.5494,
        -0.5165, -0.4928, -0.4744, -0.4658, -0.4598, -0.4682, -0.4804, -0.4992,
        -0.5257, -0.5651, -0.6205, -0.69

Progress Summary: Semi-automated data loading procedure and additional convolutional-pooling layer to the CNN, reducing the output nodes that are fed to RNN by a factor of 10. Learning rate reduced tenfold (1e-4). Run over 5 epochs shows significant improvement. Root of average MSE in last epoch is 2.07, at that instance it seems it could be further lowered by learning.
Note: Assuming values are equally distributed between -3 and 0, a random force predictor would have the metric above equal to 1.5 . Below 1.5 is needed in a test setting means so that the force predictor is relevant. Still, significant improvement must be noted.